In [1]:
import pandas as pd

In [2]:
import datetime as datetime

In [3]:
access_path = '../../data/processed/'
bal_file = 'bal_btw_rebal.dat'
# bal_file = 'balance_rebaldate_0930_all.dat'

#### 매일의 잔고(detail 테이블) INSERT

In [4]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [5]:
df_bal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-10-02,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,57760.0,115520.0,57675.0,115350.0,0.162528,NaN,NaN
1,2021-10-02,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101265.0,101265.0,101265.0,101265.0,0.142682,NaN,NaN
2,2021-10-02,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101245.0,202490.0,101245.0,202490.0,0.285308,NaN,NaN
3,2021-10-02,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104605.0,104605.0,104610.0,104610.0,0.147395,NaN,NaN
4,2021-10-02,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,108380.0,108380.0,108350.0,108350.0,0.152665,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2021-10-06,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,23.0,12875.0,296125.0,12230.0,281290.0,0.230414,NaN,NaN
356,2021-10-06,A102780,A009,적극투자형대규모,Equity,KODEX 삼성그룹,17.0,10075.0,171275.0,9580.0,162860.0,0.133404,NaN,NaN
357,2021-10-06,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,40195.0,120585.0,38260.0,114780.0,0.094020,NaN,NaN
358,2021-10-06,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,57760.0,57760.0,57515.0,57515.0,0.047112,NaN,NaN


In [6]:
df_bal.date = df_bal['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal.date = df_bal['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [7]:
df_bal.date

0      10/2/2021 4:00:00 PM
1      10/2/2021 4:00:00 PM
2      10/2/2021 4:00:00 PM
3      10/2/2021 4:00:00 PM
4      10/2/2021 4:00:00 PM
               ...         
355    10/6/2021 4:00:00 PM
356    10/6/2021 4:00:00 PM
357    10/6/2021 4:00:00 PM
358    10/6/2021 4:00:00 PM
359    10/6/2021 4:00:00 PM
Name: date, Length: 360, dtype: object

In [8]:
from sqlalchemy import create_engine

# Defining our connection variables
username = 'kdyjpzwsyqrqpb' # replace with your username
password = '6b610da9417e361c2af3be1fd0c179061b5b258f0e6f7ff9e1da464ba221e46c' # replace with your password
ipaddress = 'ec2-35-174-35-242.compute-1.amazonaws.com' # change this to your db’s IP address is if not Panoply
port = 5432 # this is the standard port for Postgres, but change it to your port if needed
dbname = 'd2ke5vkurjkusr' # change this to...you guessed it...the name of your db

# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
          
# Create the connection
cnx = create_engine(postgres_str)

In [9]:
table_nm = 'detail'
df_bal.to_sql(table_nm, con=cnx, index=False, if_exists='append')

### 자산군별 비중(general 테이블) INSERT

In [10]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [11]:
df_bal_gby = df_bal.loc[:, ['date', 'userid', 'value', 'asset_class']].sort_values(
    ['date', 'userid'],
    ascending=True).groupby(['date', 'userid', 'asset_class'
                             ]).sum().reset_index(drop=False)

In [12]:
df_bal_gby = df_bal_gby.assign(wt=df_bal_gby.groupby(['date', 'userid'])['value'].transform(lambda x: x/x.sum()))

In [13]:
df_bal_gby.date = df_bal_gby['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal_gby.date = df_bal_gby['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [14]:
df_bal_gby

,date,userid,asset_class,value,wt
0,10/2/2021 4:00:00 PM,A001,Alternative,12130.0,0.017091
1,10/2/2021 4:00:00 PM,A001,Cash,6210.0,0.008750
2,10/2/2021 4:00:00 PM,A001,Equity,59320.0,0.083582
3,10/2/2021 4:00:00 PM,A001,Fixed Income,632065.0,0.890577
4,10/2/2021 4:00:00 PM,A002,Alternative,24260.0,0.024359
...,...,...,...,...,...
175,10/6/2021 4:00:00 PM,A008,Fixed Income,57515.0,0.061061
176,10/6/2021 4:00:00 PM,A009,Alternative,300500.0,0.246149
177,10/6/2021 4:00:00 PM,A009,Cash,7460.0,0.006111
178,10/6/2021 4:00:00 PM,A009,Equity,855330.0,0.700628


In [15]:
table_nm = 'general'
df_bal_gby.to_sql(table_nm, con=cnx, index=False, if_exists='append')